# Importar Bibliotecas Necessárias
Importe as bibliotecas necessárias, incluindo PySpark.

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when

# Configurar Sessão Spark
Configure a sessão Spark para iniciar o contexto Spark.

In [3]:
# Configurar a sessão Spark com as configurações desejadas
spark = SparkSession.builder \
    .appName("Target Audience Study") \
    .config("spark.sql.shuffle.partitions", "50") \
    .config("spark.executor.memory", "2g") \
    .config("spark.driver.memory", "2g") \
    .getOrCreate()

24/11/22 20:10:31 WARN Utils: Your hostname, MacBook-Pro-de-Thales.local resolves to a loopback address: 127.0.0.1; using 192.168.1.11 instead (on interface en0)
24/11/22 20:10:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/22 20:10:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Carregar Dados em um DataFrame Spark
Carregue os dados em um DataFrame Spark a partir de uma fonte de dados, como um arquivo CSV ou um banco de dados.

In [4]:
# Especificar o diretório contendo os arquivos .parquet
parquet_directory = "/Users/zaninth/Downloads/fato_total_risco_trigg"

# Ler todos os arquivos .parquet no diretório em um único DataFrame
df = spark.read.parquet(parquet_directory)

# Mostrar o esquema do DataFrame resultante
df.printSchema()

root
 |-- ch_cpf_hash: string (nullable = true)
 |-- vl_qnt_conta: long (nullable = true)
 |-- dt_min_adesao: date (nullable = true)
 |-- dt_max_adesao: date (nullable = true)
 |-- vl_min_limite_aprovado: decimal(7,2) (nullable = true)
 |-- vl_max_limite_aprovado: decimal(7,2) (nullable = true)
 |-- vl_total_limite_aprovado: decimal(38,2) (nullable = true)
 |-- min_saldo_devedor: decimal(38,2) (nullable = true)
 |-- max_saldo_devedor: decimal(38,2) (nullable = true)
 |-- total_saldo_devedor: decimal(38,2) (nullable = true)
 |-- min_dias_atraso: integer (nullable = true)
 |-- max_dias_atraso: integer (nullable = true)
 |-- dt_fechamento: date (nullable = true)



In [20]:
# Verificar os tipos de dados de cada coluna
dtypes = df.dtypes

# Mostrar os tipos de dados
for column, dtype in dtypes:
    print(f"Coluna: {column}, Tipo de dado: {dtype}")

Coluna: ch_cpf_hash, Tipo de dado: string
Coluna: vl_qnt_conta, Tipo de dado: bigint
Coluna: dt_min_adesao, Tipo de dado: date
Coluna: dt_max_adesao, Tipo de dado: date
Coluna: vl_min_limite_aprovado, Tipo de dado: decimal(7,2)
Coluna: vl_max_limite_aprovado, Tipo de dado: decimal(7,2)
Coluna: vl_total_limite_aprovado, Tipo de dado: decimal(38,2)
Coluna: min_saldo_devedor, Tipo de dado: decimal(38,2)
Coluna: max_saldo_devedor, Tipo de dado: decimal(38,2)
Coluna: total_saldo_devedor, Tipo de dado: decimal(38,2)
Coluna: min_dias_atraso, Tipo de dado: int
Coluna: max_dias_atraso, Tipo de dado: int
Coluna: dt_fechamento, Tipo de dado: date


In [10]:
# Selecionar colunas qualitativas
qualitative_columns = ["ch_cpf_hash", "dt_fechamento"]
df_qualitative = df.select(*qualitative_columns)

# Mostrar as primeiras linhas do DataFrame qualitativo
df_qualitative.show(5)

+--------------------+-------------+
|         ch_cpf_hash|dt_fechamento|
+--------------------+-------------+
|0da564d51ee1ce79b...|   2024-08-31|
|d2a444414ab222bf5...|   2024-08-31|
|39e26353f2a9de3b0...|   2024-08-31|
|b87362d224305c9c0...|   2024-08-31|
|f9978d9d96ae87331...|   2024-08-31|
+--------------------+-------------+
only showing top 5 rows



In [18]:
# Selecionar colunas quantitativas
quantitative_columns = ["vl_qnt_conta", "vl_min_limite_aprovado", "vl_max_limite_aprovado", "vl_total_limite_aprovado", "min_saldo_devedor", "max_saldo_devedor", "total_saldo_devedor", "min_dias_atraso", "max_dias_atraso"]
df_quantitative = df.select(*quantitative_columns)

# Mostrar as primeiras linhas do DataFrame quantitativo
#df_quantitative.show(5)
# df_quantitative.describe().show() 

In [ ]:
# Converter para Pandas DataFrame para melhor visualização
df_describe_pandas = df_quantitative.describe().toPandas().convert_dtypes()

# Mostrar o DataFrame descritivo
df_describe_pandas

,summary,vl_qnt_conta,vl_min_limite_aprovado,vl_max_limite_aprovado,vl_total_limite_aprovado,min_saldo_devedor,max_saldo_devedor,total_saldo_devedor,min_dias_atraso,max_dias_atraso
0,count,1127650,1127640,1127640,1127640,1127650,1127650,1127650,1127650,1127650
1,mean,1.000527645989447,2124.722917,2125.125089,2126.672298,1270.635294,1271.083477,1271.418375,104.92518777989625,105.02991353700173
2,stddev,0.11290475029520305,1622.0309105320182,1622.3318266473307,1636.7232121960942,2404.524805272714,2404.7404514493346,2405.441601953322,232.53130138880078,232.8985853235725
3,min,1,0.00,0.00,0.00,-12464.40,-12464.40,-12464.40,0,0
4,max,54,12500.00,12500.00,96700.00,56578.72,56578.72,56578.72,2762,2762


In [30]:
df

DataFrame[ch_cpf_hash: string, vl_qnt_conta: bigint, dt_min_adesao: date, dt_max_adesao: date, vl_min_limite_aprovado: decimal(7,2), vl_max_limite_aprovado: decimal(7,2), vl_total_limite_aprovado: decimal(38,2), min_saldo_devedor: decimal(38,2), max_saldo_devedor: decimal(38,2), total_saldo_devedor: decimal(38,2), min_dias_atraso: int, max_dias_atraso: int, dt_fechamento: date]

In [31]:
import pandas_profiling as pp
import pandas as pd


# Converter para pandas DataFrame
pandas_df = df.toPandas().convert_dtypes()
pandas_df['dt_max_adesao'] = pd.to_datetime(pandas_df['dt_max_adesao'])
pandas_df['dt_min_adesao'] = pd.to_datetime(pandas_df['dt_min_adesao'])
pandas_df['dt_fechamento'] = pd.to_datetime(pandas_df['dt_fechamento'])
pandas_df['vl_min_limite_aprovado'] = pd.to_numeric(pandas_df['vl_min_limite_aprovado'].astype(float))
pandas_df['vl_max_limite_aprovado'] = pd.to_numeric(pandas_df['vl_max_limite_aprovado'].astype(float))
pandas_df['vl_total_limite_aprovado'] = pd.to_numeric(pandas_df['vl_total_limite_aprovado'].astype(float))
pandas_df['min_saldo_devedor'] = pd.to_numeric(pandas_df['min_saldo_devedor'].astype(float))
pandas_df['max_saldo_devedor'] = pd.to_numeric(pandas_df['max_saldo_devedor'].astype(float))
pandas_df['total_saldo_devedor'] = pd.to_numeric(pandas_df['total_saldo_devedor'].astype(float))

# Gerar relatório com pandas-profiling
profile = pp.ProfileReport(pandas_df)
profile.to_file("relatorio.html")


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [23]:
pandas_df.head(2)

,ch_cpf_hash,vl_qnt_conta,dt_min_adesao,dt_max_adesao,vl_min_limite_aprovado,vl_max_limite_aprovado,vl_total_limite_aprovado,min_saldo_devedor,max_saldo_devedor,total_saldo_devedor,min_dias_atraso,max_dias_atraso,dt_fechamento
0,0da564d51ee1ce79bd60bd0b723238c3878aa1d54d6da4...,1,2021-12-27,2021-12-27,600.00,600.00,600.00,307.78,307.78,307.78,149,149,2024-08-31
1,d2a444414ab222bf58b39090d7e64a66b2c1cdef2dad37...,1,2021-12-28,2021-12-28,1900.00,1900.00,1900.00,2592.25,2592.25,2592.25,220,220,2024-08-31


In [29]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col

# Resumindo a tabela
def summarize_table(df):
    print("Resumindo a tabela...")

    # Número de linhas e colunas
    num_rows = df.count()
    num_cols = len(df.columns)
    print(f" Número de Linhas: {num_rows}")
    print(f" Número de Colunas: {num_cols}")

    # Tipos de dados das colunas
    print("\n Tipos de Dados das Colunas:")
    df.printSchema()

    # Estatísticas descritivas
    print("\n Estatísticas Descritivas:")
    df.describe().show()

# Chamando a função
summarize_table(df)

Resumindo a tabela...
 Número de Linhas: 1127650
 Número de Colunas: 13

 Tipos de Dados das Colunas:
root
 |-- ch_cpf_hash: string (nullable = true)
 |-- vl_qnt_conta: long (nullable = true)
 |-- dt_min_adesao: date (nullable = true)
 |-- dt_max_adesao: date (nullable = true)
 |-- vl_min_limite_aprovado: decimal(7,2) (nullable = true)
 |-- vl_max_limite_aprovado: decimal(7,2) (nullable = true)
 |-- vl_total_limite_aprovado: decimal(38,2) (nullable = true)
 |-- min_saldo_devedor: decimal(38,2) (nullable = true)
 |-- max_saldo_devedor: decimal(38,2) (nullable = true)
 |-- total_saldo_devedor: decimal(38,2) (nullable = true)
 |-- min_dias_atraso: integer (nullable = true)
 |-- max_dias_atraso: integer (nullable = true)
 |-- dt_fechamento: date (nullable = true)


 Estatísticas Descritivas:


+-------+--------------------+-------------------+----------------------+----------------------+------------------------+-----------------+------------------+-------------------+------------------+------------------+
|summary|         ch_cpf_hash|       vl_qnt_conta|vl_min_limite_aprovado|vl_max_limite_aprovado|vl_total_limite_aprovado|min_saldo_devedor| max_saldo_devedor|total_saldo_devedor|   min_dias_atraso|   max_dias_atraso|
+-------+--------------------+-------------------+----------------------+----------------------+------------------------+-----------------+------------------+-------------------+------------------+------------------+
|  count|             1127650|            1127650|               1127640|               1127640|                 1127640|          1127650|           1127650|            1127650|           1127650|           1127650|
|   mean|                NULL|  1.000527645989447|           2124.722917|           2125.125089|             2126.672298|      1270.

In [22]:
pandas_df.dtypes

ch_cpf_hash                 string
vl_qnt_conta                 Int64
dt_min_adesao               object
dt_max_adesao               object
vl_min_limite_aprovado      object
vl_max_limite_aprovado      object
vl_total_limite_aprovado    object
min_saldo_devedor           object
max_saldo_devedor           object
total_saldo_devedor         object
min_dias_atraso              Int32
max_dias_atraso              Int32
dt_fechamento               object
dtype: object

# Explorando o DataFrame
.

In [7]:
# Obter a contagem de valores únicos na coluna 'vl_qnt_conta'
value_counts_cpf = df.groupBy("ch_cpf_hash").count().orderBy("count", ascending=False)
value_counts_qnt_con = df.groupBy("vl_qnt_conta").count().orderBy("count", ascending=False)
value_counts_min_ads = df.groupBy("dt_min_adesao").count().orderBy("count", ascending=False)
value_counts_max_ads = df.groupBy("dt_max_adesao").count().orderBy("count", ascending=False)
val_cnt_min_lmt_apr = df.groupBy("vl_min_limite_aprovado").count().orderBy("count", ascending=False)
vl_max_limite_aprovado = df.groupBy("vl_max_limite_aprovado").count().orderBy("count", ascending=False)
vl_total_limite_aprovado = df.groupBy("vl_total_limite_aprovado").count().orderBy("count", ascending=False)
min_saldo_devedor = df.groupBy("min_saldo_devedor").count().orderBy("count", ascending=False)
max_saldo_devedor = df.groupBy("max_saldo_devedor").count().orderBy("count", ascending=False)
total_saldo_devedor = df.groupBy("total_saldo_devedor").count().orderBy("count", ascending=False)
min_dias_atraso = df.groupBy("min_dias_atraso").count().orderBy("count", ascending=False)
max_dias_atraso = df.groupBy("max_dias_atraso").count().orderBy("count", ascending=False)
dt_fechamento = df.groupBy("dt_fechamento").count().orderBy("count", ascending=False)

# Mostrar o resultado
value_counts_cpf.show()
value_counts_qnt_con.show()
value_counts_min_ads.show()
value_counts_max_ads.show()
val_cnt_min_lmt_apr.show()
vl_max_limite_aprovado.show()
vl_total_limite_aprovado.show()
min_saldo_devedor.show()
max_saldo_devedor.show()
total_saldo_devedor.show()
min_dias_atraso.show()
max_dias_atraso.show()
dt_fechamento.show()

+--------------------+-----+
|         ch_cpf_hash|count|
+--------------------+-----+
|038ffd39fb58c5f1b...|    5|
|3d79d5e702a97715c...|    5|
|0c0166d66319820fc...|    5|
|9806dde903c8a1bfa...|    5|
|86954a39d2a775d17...|    5|
|fd9761d637c3ce969...|    5|
|98199d258b3cbf426...|    5|
|1a08db347c20a4384...|    5|
|4286661ba0a8c231e...|    5|
|8bbeb0febfd217152...|    5|
|ebd93149f5719b433...|    5|
|23fbbd327853c8039...|    5|
|69a88a3fc9f492197...|    5|
|9ecee4185f4bdf0e4...|    5|
|7e676758c856f3512...|    5|
|264a1aa26619c3782...|    5|
|a206f6d3f06094a98...|    5|
|7d368838ce524dc4b...|    5|
|bce661a71eaf5bec7...|    5|
|aecb26c555bdea77d...|    5|
+--------------------+-----+
only showing top 20 rows

+------------+-------+
|vl_qnt_conta|  count|
+------------+-------+
|           1|1127315|
|           2|    330|
|          54|      5|
+------------+-------+

+-------------+-----+
|dt_min_adesao|count|
+-------------+-----+
|   2022-08-17| 3740|
|   2021-01-04| 3675|
|   20

+-----------------+------+
|min_saldo_devedor| count|
+-----------------+------+
|             0.00|167362|
|            13.90|  8993|
|             0.02|  7502|
|             0.04|  5345|
|             0.06|  5061|
|             0.03|  4402|
|             0.01|  4119|
|             0.05|  3882|
|             0.07|  2942|
|            -0.01|  2899|
|             0.08|  2708|
|             0.10|  2166|
|             0.09|  2120|
|            -0.50|  2007|
|            -1.00|  1989|
|            23.89|  1808|
|             0.11|  1519|
|             0.12|  1416|
|             0.14|  1409|
|             0.66|  1339|
+-----------------+------+
only showing top 20 rows

+-----------------+------+
|max_saldo_devedor| count|
+-----------------+------+
|             0.00|167331|
|            13.90|  8992|
|             0.02|  7492|
|             0.04|  5350|
|             0.06|  5061|
|             0.03|  4402|
|             0.01|  4124|
|             0.05|  3887|
|             0.07|  2947|
| 

In [9]:
# Mostrar as primeiras linhas do DataFrame
df.show(5)

# Mostrar o número de linhas e colunas do DataFrame
num_rows = df.count()
num_cols = len(df.columns)
print(f"O DataFrame concatenado tem {num_rows} linhas e {num_cols} colunas.")

+--------------------+------------+-------------+-------------+----------------------+----------------------+------------------------+-----------------+-----------------+-------------------+---------------+---------------+-------------+
|         ch_cpf_hash|vl_qnt_conta|dt_min_adesao|dt_max_adesao|vl_min_limite_aprovado|vl_max_limite_aprovado|vl_total_limite_aprovado|min_saldo_devedor|max_saldo_devedor|total_saldo_devedor|min_dias_atraso|max_dias_atraso|dt_fechamento|
+--------------------+------------+-------------+-------------+----------------------+----------------------+------------------------+-----------------+-----------------+-------------------+---------------+---------------+-------------+
|0da564d51ee1ce79b...|           1|   2021-12-27|   2021-12-27|                600.00|                600.00|                  600.00|           307.78|           307.78|             307.78|            149|            149|   2024-08-31|
|d2a444414ab222bf5...|           1|   2021-12-28|   

# Transformações e Ações em DataFrame Spark
Execute transformações e ações em DataFrames Spark para manipular e analisar os dados.

In [10]:
# Remover linhas com valores nulos em colunas importantes
df_clean = df.dropna(subset=["ch_cpf_hash", "dt_fechamento", "vl_qnt_conta"])

# Verificar novamente o número de linhas após a limpeza
num_rows_clean = df_clean.count()
print(f"O DataFrame limpo tem {num_rows_clean} linhas.")

O DataFrame limpo tem 1127650 linhas.


# Salvar Resultados
Salve os resultados das operações em um formato de arquivo ou banco de dados.

In [12]:
# Supondo que existam colunas como 'idade' e 'sexo' no DataFrame
# Calcular a distribuição de idade
df_clean.groupBy("vl_min_limite_aprovado").count().orderBy("vl_min_limite_aprovado").show()

# Calcular a distribuição de sexo
df_clean.groupBy("vl_qnt_conta").count().show()

+----------------------+-----+
|vl_min_limite_aprovado|count|
+----------------------+-----+
|                  NULL|   10|
|                  0.00|   45|
|                100.00|   15|
|                300.00|    5|
|                500.00|39035|
|                600.00|85125|
|                700.00|29610|
|                727.21|    5|
|                800.00|25515|
|                841.42|    5|
|                900.00|31705|
|                915.53|    5|
|                930.88|    5|
|                944.51|    5|
|                945.58|    5|
|                950.00|    5|
|                972.00|    5|
|               1000.00|44965|
|               1035.93|    5|
|               1040.48|    5|
+----------------------+-----+
only showing top 20 rows

+------------+-------+
|vl_qnt_conta|  count|
+------------+-------+
|          54|      5|
|           1|1127315|
|           2|    330|
+------------+-------+



In [13]:
# Criar segmentos com base em critérios específicos, por exemplo, idade e saldo devedor
df_segmented = df_clean.withColumn(
    "segmento",
    when((col("idade") < 30) & (col("total_saldo_devedor") < 1000), "Jovens com baixo saldo devedor")
    .when((col("idade") >= 30) & (col("total_saldo_devedor") >= 1000), "Adultos com alto saldo devedor")
    .otherwise("Outros")
)

# Mostrar a distribuição dos segmentos
df_segmented.groupBy("segmento").count().show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `idade` cannot be resolved. Did you mean one of the following? [`dt_max_adesao`, `dt_min_adesao`, `ch_cpf_hash`, `dt_fechamento`, `min_dias_atraso`].;
'Project [ch_cpf_hash#135, vl_qnt_conta#136L, dt_min_adesao#137, dt_max_adesao#138, vl_min_limite_aprovado#139, vl_max_limite_aprovado#140, vl_total_limite_aprovado#141, min_saldo_devedor#142, max_saldo_devedor#143, total_saldo_devedor#144, min_dias_atraso#145, max_dias_atraso#146, dt_fechamento#147, CASE WHEN (('idade < 30) AND (total_saldo_devedor#144 < cast(cast(1000 as decimal(4,0)) as decimal(38,2)))) THEN Jovens com baixo saldo devedor WHEN (('idade >= 30) AND (total_saldo_devedor#144 >= cast(cast(1000 as decimal(4,0)) as decimal(38,2)))) THEN Adultos com alto saldo devedor ELSE Outros END AS segmento#335]
+- Filter atleastnnonnulls(3, ch_cpf_hash#135, dt_fechamento#147, vl_qnt_conta#136L)
   +- Relation [ch_cpf_hash#135,vl_qnt_conta#136